In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer

#defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'rachaeld-data445'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
##defining input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

##splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)

In [4]:
X_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3352.000000,3374.000000,3389.000000
mean,49.575811,0.499705,236.459726,25.803269,75.622603
std,8.519847,0.500074,44.272381,4.084957,11.926481
min,33.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,205.000000,23.070000,68.000000
50%,49.000000,0.000000,233.000000,25.420000,75.000000
75%,56.000000,1.000000,263.000000,28.040000,82.000000
max,70.000000,1.000000,600.000000,56.800000,130.000000


In [5]:
X_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,836.000000,845.00000,848.000000
mean,49.621462,0.471698,237.771531,25.79697,76.903302
std,8.783201,0.499493,45.855088,4.06311,12.372899
min,32.000000,0.000000,145.000000,15.96000,50.000000
25%,42.000000,0.000000,207.000000,23.09000,68.000000
50%,49.000000,0.000000,235.000000,25.23000,75.000000
75%,57.000000,1.000000,262.000000,27.98000,85.000000
max,69.000000,1.000000,696.000000,43.69000,143.000000


In [6]:
##defining the imputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer.fit(X_train)

##imputing the missing values
X_train_imp = pd.DataFrame(imputer.transform(X_train), columns = X_train.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)

In [7]:
X_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.575811,0.499705,236.420944,25.801460,75.622419
std,8.519847,0.500074,44.024981,4.075388,11.924726
min,33.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.070000,68.000000
50%,49.000000,0.000000,233.000000,25.420000,75.000000
75%,56.000000,1.000000,263.000000,28.040000,82.000000
max,70.000000,1.000000,600.000000,56.800000,130.000000


In [8]:
X_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.621462,0.471698,237.704009,25.795637,76.903302
std,8.783201,0.499493,45.532592,4.055970,12.372899
min,32.000000,0.000000,145.000000,15.960000,50.000000
25%,42.000000,0.000000,207.000000,23.090000,68.000000
50%,49.000000,0.000000,235.000000,25.260000,75.000000
75%,57.000000,1.000000,261.000000,27.980000,85.000000
max,69.000000,1.000000,696.000000,43.690000,143.000000


In [10]:
scaler = MinMaxScaler()
X_train_imp = pd.DataFrame(scaler.fit_transform(X_train_imp), columns = X_train_imp.columns)
X_test_imp = pd.DataFrame(scaler.fit_transform(X_test_imp), columns = X_test_imp.columns)

In [12]:
### LOGISTIC REGRESSION ###
logit_md = LogisticRegression().fit(X_train_imp, Y_train)

##predicting on test
logit_pred = logit_md.predict_proba(X_test_imp)[:,1]

##changing liklihoods to labels
logit_label = np.where(logit_pred < .1, 0, 1)

print(classification_report(Y_test, logit_label))

              precision    recall  f1-score   support

           0       0.97      0.36      0.53       719
           1       0.21      0.93      0.34       129

    accuracy                           0.45       848
   macro avg       0.59      0.65      0.43       848
weighted avg       0.85      0.45      0.50       848



In [14]:
### RANDOM FOREST ###
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_imp, Y_train)

##predicting on test
RF_pred = RF_md.predict_proba(X_test_imp)[:,1]

##changing liklihoods to labels
RF_label = np.where(RF_pred < .1, 0, 1)

print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.95      0.33      0.49       719
           1       0.19      0.90      0.32       129

    accuracy                           0.42       848
   macro avg       0.57      0.61      0.40       848
weighted avg       0.83      0.42      0.46       848



In [ ]:
##from my above results we can see that I would use logistic regression to predict TenYearCHD because it has overall better scores.